In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'c:\\New_Project'

In [5]:
!nvidia-smi

Sun Nov  2 01:45:08 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.07                 Driver Version: 566.07         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3050 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   57C    P8              9W /   35W |     144MiB /   4096MiB |     38%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("Device count:", torch.cuda.device_count())
print("Current device:", torch.cuda.current_device())
print("Device name:", torch.cuda.get_device_name(0))

In [ ]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model: Path
    num_labels: int
    num_train_epochs: int
    per_device_train_batch_size: int
    per_device_eval_batch_size: int
    gradient_accumulation_steps: int
    weight_decay: float
    learning_rate: float
    eval_strategy: str
    save_strategy: str
    load_best_model_at_end: bool
    remove_unused_columns: bool
    optim: str
    number_of_unfreeze_layers : int

In [ ]:
from src.DocumindAI.constants import *
from src.DocumindAI.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model = config.model,
            num_labels = params.num_labels,
            num_train_epochs = params.num_train_epochs,
            per_device_train_batch_size = params.per_device_train_batch_size,
            per_device_eval_batch_size = params.per_device_eval_batch_size,
            learning_rate = params.learning_rate,
            eval_strategy = params.eval_strategy,
            save_strategy = params.save_strategy,
            load_best_model_at_end = params.load_best_model_at_end,
            remove_unused_columns = params.remove_unused_columns,
            gradient_accumulation_steps = params.gradient_accumulation_steps,
            weight_decay = params.weight_decay,
            optim = params.optim,
            number_of_unfreeze_layers = params.number_of_unfreeze_layers
        )

        return model_trainer_config

In [ ]:
from datasets import load_from_disk
from transformers import LayoutLMv2ForSequenceClassification
from transformers import TrainingArguments, Trainer, AutoProcessor

In [ ]:
class ModelTrainer:
    def __init__(self, config:ModelTrainerConfig):
        self.config = config
        self.model = None
        self.preprocessor = AutoProcessor.from_pretrained(self.config.model,apply_ocr=True)

    def load_encoded_dataset(self):
        print("Loading encoded dataset from disk")
        base_path = self.config.data_path

        self.encoded_dataset = {
            'train': load_from_disk(os.path.join(base_path, "train")),
            'val': load_from_disk(os.path.join(base_path, "val")),
            'test': load_from_disk(os.path.join(base_path, "test"))
        }

        for split_name in self.encoded_dataset:
            self.encoded_dataset[split_name].set_format(type='torch')

        print("✅ Encoded dataset successfully loaded and formatted!")

    def initialize_model(self):
        print("Initializing LayoutLMv2 model")
        model_name = self.config.model
        num_labels = self.config.num_labels

        self.model = LayoutLMv2ForSequenceClassification.from_pretrained(
            model_name,
            num_labels=num_labels
        )

        print("✅ Model initialized successfully!")

    def unfreeze_layers(self):
        model_name = self.model

        for param in model_name.parameters():
            param.requires_grad = False

        for param in model_name.classifier.parameters():
            param.requires_grad = True

        number_of_unfreeze_layers = self.config.number_of_unfreeze_layers
        pass_value = -1*number_of_unfreeze_layers
        for param in model_name.layoutlmv3.encoder.layer[pass_value:].parameters():
            param.requires_grad = True        

    def setup_trainer(self):
        print("Creating Trainer instance")
        self.training_args = TrainingArguments(
            num_train_epochs=self.config.num_train_epochs,
            per_device_train_batch_size=self.config.per_device_train_batch_size,
            per_device_eval_batch_size=self.config.per_device_eval_batch_size,
            gradient_accumulation_steps = self.config.gradient_accumulation_steps,
            weight_decay = self.config.weight_decay,
            learning_rate=self.config.learning_rate,
            eval_strategy=self.config.eval_strategy,
            save_strategy=self.config.save_strategy,
            load_best_model_at_end=self.config.load_best_model_at_end,
            remove_unused_columns=self.config.remove_unused_columns,
            optim = self.config.optim,
            report_to=None
        )
        self.trainer = Trainer(
            model=self.model,
            args=self.training_args,
            train_dataset=self.encoded_dataset["train"],
            eval_dataset=self.encoded_dataset["val"],
        )
        print("Trainer ready!")

    def train_model(self):
        print("Starting model training...")
        self.trainer.train()
        print("Training completed!")

    def save_model(self):
        model_dir = os.path.join(self.config.root_dir,"documind_model")
        os.makedirs(model_dir, exist_ok=True)

        print(f"Saving model to: {model_dir}")
        self.preprocessor.save_pretrained(model_dir)
        self.trainer.save_model(model_dir)
        print("Model and trainer saved successfully!")

    def train(self):
        print("Running full model training pipeline")
        self.load_encoded_dataset()
        self.initialize_model()
        self.unfreeze_layers()
        self.setup_trainer()
        self.train_model()
        self.save_model()
        print(" Model training pipeline completed successfully!")


In [ ]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e